<a href="https://colab.research.google.com/github/ramkumarr02/Titanic/blob/master/Titanic_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.executing_eagerly()

print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


# Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Deep Learning/Titanic/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Deep Learning/Titanic/test.csv')

train_copy = train.copy
test_copy = test.copy

# Functions

In [0]:
def column_split(df):

    df['SurName'], df['Name'] = df['Name'].str.split(',', 1).str
    df['Title'], df['Name'] = df['Name'].str.split('.', 1).str
    df['Cabin_Section'] = df[df['Cabin'].notna()]['Cabin'].astype(str).str[0]
    df['Cabin_Nums'] = df[df['Cabin'].notna()]['Cabin'].str.count(" ") + 1
    
    return(df)

In [0]:
def encode_str(df):

    df = pd.get_dummies(df, columns=['Sex', 'Embarked','Title','Cabin_Section'])
    df = df.round(1)

    text_cols = list(df.select_dtypes(include=['object']).columns)
    df = df.drop(text_cols, axis=1)
    
    return(df)

In [0]:
def data_preprocess(df):

    df = column_split(df)
    df = encode_str(df)
    df.fillna(0, inplace = True)
 
    return(df)

# Code Engine

In [0]:
df = data_preprocess(train)

In [0]:
target = df.pop('Survived')
scaled_features = StandardScaler().fit_transform(df.values)
df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
train_dataset = dataset.shuffle(len(df)).batch(1)

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',    
    metrics=['accuracy'])

In [23]:
model.fit(train_dataset, epochs=5)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/5
892/892 [==============================] - 6s 6ms/step - loss: 0.5064 - accuracy: 0.7915
Epoch 2/5
892/892 [==============================] - 2s 2ms/step - loss: 0.4190 - accuracy: 0.8341
Epoch 3/5
892/892 [==============================] - 2s 2ms/step - loss: 0.4103 - accuracy: 0.8307
Epoch 4/5
892/892 [==============================] - 2s 2ms/step - loss: 0.4076 - accuracy: 0.8330
Epoch 5/5
892/892 [==============================] - 2s 2ms/step - loss: 0.3937 - accuracy: 0.8296
